In [3]:
#Youtube like/view rate prediction

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from flaml import AutoML
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings(action='ignore')

c:\Users\dogucan.elci\Miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_df = pd.read_parquet('train.parquet')
test_df = pd.read_parquet('test.parquet')
sample_submission = pd.read_csv('sample_submission.csv')

In [6]:
len(train_df) , train_df.video_id.nunique()

(92275, 16969)

In [7]:
train_df[train_df.video_id.map(train_df.video_id.value_counts()>20)].head(2)
#videos which keep to be include as trends for minimum 20 days

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,id,duration_seconds,has_thumbnail,target
7400,pvPsJFRGleA,Justin Bieber - Holy ft. Chance The Rapper,2020-09-18 04:00:10+00:00,UCHkj014U2CQ2Nv0UZeYpE_A,JustinBieberVEVO,10,2020-09-18,Justin|Bieber|Holy|RBMG/Def|Jam|Pop,6217404,936304,13331,91366,https://i.ytimg.com/vi/pvPsJFRGleA/default.jpg,False,False,Holy out now: https://JustinBieber.lnk.to/Holy...,pvPsJFRGleA_2020-09-18,329.0,False,0.150594
7601,pvPsJFRGleA,Justin Bieber - Holy ft. Chance The Rapper,2020-09-18 04:00:10+00:00,UCHkj014U2CQ2Nv0UZeYpE_A,JustinBieberVEVO,10,2020-09-19,justin bieber|bieber|justin|hailey|biebs|biebe...,16239716,1555576,29634,134954,https://i.ytimg.com/vi/pvPsJFRGleA/default.jpg,False,False,Holy out now: https://JustinBieber.lnk.to/Holy...,pvPsJFRGleA_2020-09-19,329.0,False,0.095788


In [8]:
train_df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,id,duration_seconds,has_thumbnail,target
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11 19:20:14+00:00,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,3C66w5Z0ixs_2020-08-12,NaN,True,0.103596


In [9]:
train_df.iloc[1]

video_id                                                   M9Pmf9AB4Mo
title                Apex Legends | Stories from the Outlands – “Th...
publishedAt                                  2020-08-11 17:00:10+00:00
channelId                                     UC0ZV6M2THA81QT9hrVWJG3A
channelTitle                                              Apex Legends
categoryId                                                          20
trending_date                                               2020-08-12
tags                 Apex Legends|Apex Legends characters|new Apex ...
view_count                                                     2381688
likes                                                           146739
dislikes                                                          2794
comment_count                                                    16549
thumbnail_link          https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg
comments_disabled                                                False
rating

In [10]:
test_df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,thumbnail_link,comments_disabled,ratings_disabled,description,id,duration_seconds,has_thumbnail,isTest
0,_wNsZEqpKUA,"Billie Eilish: Same Interview, The Fifth Year ...",2021-11-30 17:00:10+00:00,UCIsbLox_y9dCIMLd8tdC6qg,Vanity Fair,24,2021-12-01,billie|billie eilish|billie eilish 2021|billie...,https://i.ytimg.com/vi/_wNsZEqpKUA/default.jpg,False,False,I'm so much more open to stuff now. On the ver...,_wNsZEqpKUA_2021-12-01,1335.0,True,True


In [11]:
#different columns between train and test dataset
set(train_df.columns) - set(test_df.columns)

{'comment_count', 'dislikes', 'likes', 'target', 'view_count'}

Check if any unconvenient dtypes exists

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92275 entries, 0 to 92274
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   video_id           92275 non-null  object             
 1   title              92275 non-null  object             
 2   publishedAt        92275 non-null  datetime64[ns, UTC]
 3   channelId          92275 non-null  object             
 4   channelTitle       92275 non-null  object             
 5   categoryId         92275 non-null  int64              
 6   trending_date      92275 non-null  object             
 7   tags               92275 non-null  object             
 8   view_count         92275 non-null  int64              
 9   likes              92275 non-null  int64              
 10  dislikes           92275 non-null  int64              
 11  comment_count      92275 non-null  int64              
 12  thumbnail_link     92275 non-null  object     

trending_date is obejct,convert it to datetime dtype format

In [13]:
train_df.trending_date = pd.to_datetime(train_df.trending_date)
train_df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,id,duration_seconds,has_thumbnail,target
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11 19:20:14+00:00,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,3C66w5Z0ixs_2020-08-12,NaN,True,0.103596


In [14]:
test_df.trending_date = pd.to_datetime(test_df.trending_date)
test_df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,thumbnail_link,comments_disabled,ratings_disabled,description,id,duration_seconds,has_thumbnail,isTest
0,_wNsZEqpKUA,"Billie Eilish: Same Interview, The Fifth Year ...",2021-11-30 17:00:10+00:00,UCIsbLox_y9dCIMLd8tdC6qg,Vanity Fair,24,2021-12-01,billie|billie eilish|billie eilish 2021|billie...,https://i.ytimg.com/vi/_wNsZEqpKUA/default.jpg,False,False,I'm so much more open to stuff now. On the ver...,_wNsZEqpKUA_2021-12-01,1335.0,True,True


check if any nan values exists for all columns

In [15]:
train_df.isna().sum()

video_id                0
title                   0
publishedAt             0
channelId               0
channelTitle            0
categoryId              0
trending_date           0
tags                    0
view_count              0
likes                   0
dislikes                0
comment_count           0
thumbnail_link          0
comments_disabled       0
ratings_disabled        0
description          1476
id                      0
duration_seconds     2176
has_thumbnail           0
target                  0
dtype: int64

In [16]:
train_df.isna().sum()['duration_seconds'] / len(train_df)

0.023581685180167976

In [17]:
train_df.dropna(subset='duration_seconds',inplace=True)

In [18]:
test_df.isna().sum()['duration_seconds'] / len(test_df)

0.0034482758620689655

In [19]:
test_df.dropna(subset='duration_seconds',inplace=True)

Check any category exists in test data but not exists in train data, it is counted as outier data

In [20]:
set(test_df.categoryId.unique()) - set(train_df.categoryId.unique())

set()

If ratings are disabled, it is not possible to predict target(like_count/total_review).So these must be removed.

In [21]:
display(len(train_df[train_df.ratings_disabled == True]))
train_df = train_df[train_df.ratings_disabled == False]

431

In [22]:
display(len(test_df[test_df.ratings_disabled == True]))
test_df = test_df[test_df.ratings_disabled == False]

215

In [23]:
len(test_df) , len(train_df)

(5565, 89668)

Encoding Part
Some boolean columns are convert to numeric type and some wide range columns are converted to 0 to n range.

In [24]:
from sklearn import preprocessing 
encoder = preprocessing.LabelEncoder()

train_df['categoryId'] = encoder.fit_transform(train_df.categoryId)
test_df['categoryId'] = encoder.transform(test_df.categoryId)

In [25]:
encoder_boolean = preprocessing.LabelEncoder()

train_df['comments_disabled'] = encoder_boolean.fit_transform(train_df.comments_disabled)
test_df['comments_disabled'] = encoder_boolean.transform(test_df.comments_disabled)

In [26]:
encoder_boolean1 = preprocessing.LabelEncoder()

train_df['has_thumbnail'] = encoder_boolean1.fit_transform(train_df.has_thumbnail)
test_df['has_thumbnail'] = encoder_boolean1.transform(test_df.has_thumbnail)

Feature Engineering

1- Time Features

In [27]:
train_df.publishedAt.dt.dayofweek

40       1
41       1
42       1
43       2
44       1
        ..
92270    1
92271    2
92272    1
92273    6
92274    2
Name: publishedAt, Length: 89668, dtype: int64

In [28]:
train_df['publish_day'] = train_df.publishedAt.dt.dayofweek
train_df['publish_hour'] = train_df.publishedAt.dt.hour
train_df['current_day'] = train_df.trending_date.dt.dayofweek

In [29]:
test_df['publish_day'] = test_df.publishedAt.dt.dayofweek
test_df['publish_hour'] = test_df.publishedAt.dt.hour
test_df['current_day'] = test_df.trending_date.dt.dayofweek

In [30]:
train_df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,...,comments_disabled,ratings_disabled,description,id,duration_seconds,has_thumbnail,target,publish_day,publish_hour,current_day
40,n_8hxS_7Vis,Shannon reacts to Kyle Kuzma's game-winning sh...,2020-08-11 16:21:47+00:00,UCLXzq85ijg2LwJWFrz4pkmw,Skip and Shannon: UNDISPUTED,4,2020-08-12,fox|fox sports|fs1|fox sports 1|undisputed|ski...,540613,7155,...,0,False,LeBron and the Lakers finally snapped their 3-...,n_8hxS_7Vis_2020-08-12,1112.0,1,0.013235,1,16,2


In [31]:
test_df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,thumbnail_link,comments_disabled,ratings_disabled,description,id,duration_seconds,has_thumbnail,isTest,publish_day,publish_hour,current_day
0,_wNsZEqpKUA,"Billie Eilish: Same Interview, The Fifth Year ...",2021-11-30 17:00:10+00:00,UCIsbLox_y9dCIMLd8tdC6qg,Vanity Fair,9,2021-12-01,billie|billie eilish|billie eilish 2021|billie...,https://i.ytimg.com/vi/_wNsZEqpKUA/default.jpg,0,False,I'm so much more open to stuff now. On the ver...,_wNsZEqpKUA_2021-12-01,1335.0,1,True,1,17,2


2-Total trending days

In [32]:
train_df['total_trend_days'] = (train_df.trending_date.dt.date - train_df.publishedAt.dt.date).dt.days
test_df['total_trend_days'] = (test_df.trending_date.dt.date - test_df.publishedAt.dt.date).dt.days

3-target-based Features:<br>
- channel_average_targets<br>
- category_average_targets

In [33]:
train_df['channel_average_target'] = train_df.groupby(['channelId'])['target'].transform('mean')
train_df['categoryId_average_target'] = train_df.groupby(['categoryId'])['target'].transform('mean')

In [34]:
test_df = test_df.merge(train_df.loc[:,['categoryId','categoryId_average_target']].drop_duplicates(subset='categoryId'),on='categoryId',how='left')

In [35]:
test_df = test_df.merge(train_df.loc[:,['channelId','channel_average_target']].drop_duplicates(subset='channelId'),on='channelId',how='left')

In [41]:
test_df.channel_average_target.fillna(test_df.categoryId_average_target, inplace=True)

In [43]:
test_df.isna().sum()

video_id                       0
title                          0
publishedAt                    0
channelId                      0
channelTitle                   0
categoryId                     0
trending_date                  0
tags                           0
thumbnail_link                 0
comments_disabled              0
ratings_disabled               0
description                  385
id                             0
duration_seconds               0
has_thumbnail                  0
isTest                         0
publish_day                    0
publish_hour                   0
current_day                    0
total_trend_days               0
categoryId_average_target      0
channel_average_target         0
dtype: int64

4-popular_tag_count

Popular tags

In [48]:
train_df.tags.str.split('|',expand=True).stack().value_counts()

[None]                         13062
funny                           5820
comedy                          3758
minecraft                       3151
challenge                       2324
                               ...  
nile wilson                        1
official strongman                 1
hathor bjornsson                   1
giants live                        1
I was in MrBeast Squid Game        1
Length: 111818, dtype: int64

In [51]:
top_100_popular_tags = train_df.tags.str.split('|',expand=True).stack().value_counts()[:100].keys()
top_100_popular_tags

Index(['[None]', 'funny', 'comedy', 'minecraft', 'challenge', 'vlog', 'news',
       'animation', 'rap', 'music', 'gaming', 'highlights', 'tiktok', 'how to',
       'reaction', 'football', 'hip hop', 'family', '2020', 'new',
       'family friendly', 'game', 'fortnite', 'video', 'sports', 'Rap', 'fun',
       'tik tok', 'NBA', '2021', 'Minecraft', 'Football', 'review', 'soccer',
       'music video', 'Records', 'Hip Hop', 'minecraft challenge', 'trailer',
       'dream', 'among us', 'diy', 'science', 'gameplay', 'parody',
       'interview', 'play', 'nba', 'live', 'tutorial', 'Music', 'reacting',
       'Pop', 'espn', 'Basketball', 'marvel', 'boxing', 'cartoon', 'meme',
       'NFL', 'highlight', 'laugh', 'mine craft', 'humor', 'BTS',
       'minecraft but', 'education', 'Highlights', 'yt:cc=on', 'basketball',
       'experiment', 'entertainment', 'shorts', 'nintendo', 'food', 'prank',
       'lil durk', 'season', 'hilarious', 'G League', 'viral',
       'fortnite battle royale', 'matp

In [65]:
len(set(train_df.tags.iloc[0].split('|')) & set(top_100_popular_tags))

4

In [68]:
def popular_tag_count(x):
    count = len(set(x.tags.split('|')) & set(top_100_popular_tags))
    return count

In [69]:
train_df['popular_tag_count'] = train_df.apply(lambda x: popular_tag_count(x),axis=1)

In [71]:
test_df['popular_tag_count'] = test_df.apply(lambda x: popular_tag_count(x),axis=1)

Modeling

In [75]:
train_df.columns

Index(['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'categoryId', 'trending_date', 'tags', 'view_count', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'description', 'id', 'duration_seconds',
       'has_thumbnail', 'target', 'publish_day', 'publish_hour', 'current_day',
       'total_trend_days', 'channel_average_target',
       'categoryId_average_target', 'popular_tag_count'],
      dtype='object')

In [82]:
train_df_filtered = train_df.loc[:,['categoryId','comments_disabled',
'duration_seconds','has_thumbnail','publish_day','publish_hour','current_day',
'total_trend_days','channel_average_target','categoryId_average_target','popular_tag_count','target']]

In [83]:
test_df_filtered = test_df.loc[:,['categoryId','comments_disabled',
'duration_seconds','has_thumbnail','publish_day','publish_hour','current_day',
'total_trend_days','channel_average_target','categoryId_average_target','popular_tag_count']]

In [88]:
#Train and evaluation datas are seperated.
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_df_filtered.drop(columns=['target']), train_df_filtered.loc[:,['target']], test_size=0.2, random_state=40)

In [89]:
#AutoML function is useful for detecting most optimal model as its metric,task and time_budget.
#After finishing whole iteration of this function, it is possible to learn best estimator and its best configurations.
automl3 = AutoML()
settings = {
    "time_budget": 500,  
    "metric": 'mae',
    "estimator_list": ['lgbm','xgboost','catboost'],
    "task": 'regression',
}
automl3.fit(X_train=X_train, y_train=y_train.values, **settings)

[flaml.automl: 08-31 02:09:03] {2565} INFO - task = regression
[flaml.automl: 08-31 02:09:03] {2567} INFO - Data split method: uniform
[flaml.automl: 08-31 02:09:03] {2570} INFO - Evaluation method: cv
[flaml.automl: 08-31 02:09:03] {2689} INFO - Minimizing error metric: mae
[flaml.automl: 08-31 02:09:03] {2831} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'catboost']
[flaml.automl: 08-31 02:09:03] {3133} INFO - iteration 0, current learner lgbm
[flaml.automl: 08-31 02:09:03] {3266} INFO - Estimated sufficient time budget=2709s. Estimated necessary time budget=5s.
[flaml.automl: 08-31 02:09:03] {3313} INFO -  at 0.4s,	estimator lgbm's best error=0.0222,	best estimator lgbm's best error=0.0222
[flaml.automl: 08-31 02:09:03] {3133} INFO - iteration 1, current learner lgbm
[flaml.automl: 08-31 02:09:03] {3313} INFO -  at 0.7s,	estimator lgbm's best error=0.0222,	best estimator lgbm's best error=0.0222
[flaml.automl: 08-31 02:09:03] {3133} INFO - iteration 2, current learn

In [90]:
#best estimator is lgbm
automl3.best_estimator

'lgbm'

In [91]:
lgbm_best_config = automl3.best_config
lgbm_best_config

{'n_estimators': 167,
 'num_leaves': 1270,
 'min_child_samples': 2,
 'learning_rate': 0.24828517320539853,
 'log_max_bin': 10,
 'colsample_bytree': 0.6155762959638018,
 'reg_alpha': 0.007542890796625709,
 'reg_lambda': 0.005445061815445535}

In [96]:
from lightgbm import LGBMRegressor
model_lgbm = LGBMRegressor(**lgbm_best_config)
model_lgbm.fit(X_train, y_train)

LGBMRegressor(colsample_bytree=0.6155762959638018,
              learning_rate=0.24828517320539853, log_max_bin=10,
              min_child_samples=2, n_estimators=167, num_leaves=1270,
              reg_alpha=0.007542890796625709, reg_lambda=0.005445061815445535)

In [97]:
preds = model_lgbm.predict(X_val)

In [99]:
mean_absolute_error(y_val,preds)

0.0040886692145699725